## Observations and Insights 

In [5]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np

# Study data files
mouse_metadata_path = "Data/Mouse_metadata.csv"
study_results_path = "Data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)


# Combine the data into a single dataset

df_merge=pd.DataFrame.merge(mouse_metadata,study_results, on="Mouse ID", how="outer", left_index=False, right_index=False)



# Display the data table for preview
df_merge



,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1
...,...,...,...,...,...,...,...,...
1888,z969,Naftisol,Male,9,30,25,63.145652,2
1889,z969,Naftisol,Male,9,30,30,65.841013,3
1890,z969,Naftisol,Male,9,30,35,69.176246,4
1891,z969,Naftisol,Male,9,30,40,70.314904,4


In [6]:
# Checking the number of mice.

df_merge["Mouse ID"].nunique()


249

In [7]:
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 

df2= df_merge.loc[df_merge.duplicated(subset=['Mouse ID', 'Timepoint']), 'Mouse ID'].unique()
df2



array(['g989'], dtype=object)

In [232]:
# Optional: Get all the data for the duplicate mouse ID. 


In [8]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.


df=df_merge[df_merge["Mouse ID"] != 'g989']



In [9]:
# Checking the number of mice in the clean DataFrame
df["Mouse ID"].nunique()

248

## Summary Statistics

In [10]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

tumor_mean=df.groupby(["Drug Regimen"])["Tumor Volume (mm3)"].mean()
tumor_median=df.groupby(["Drug Regimen"])["Tumor Volume (mm3)"].median()
tumor_std=df.groupby(["Drug Regimen"])["Tumor Volume (mm3)"].std()
tumor_sem=df.groupby(["Drug Regimen"])["Tumor Volume (mm3)"].sem()

# # df.dtypes

# # This method is the most straighforward, creating multiple series and putting them all together at the end.

mean_df=pd.DataFrame({"Tumor Mean (mm3)":tumor_mean})
median_df=pd.DataFrame({"Tumor Median (mm3)":tumor_median})
std_df=pd.DataFrame({"Tumor STD (mm3)": tumor_std})
sem_df=pd.DataFrame({"Tumor SEM (mm3)": tumor_sem})
mean_median=pd.DataFrame.merge(mean_df,median_df, on="Drug Regimen")
std_sem=pd.DataFrame.merge(std_df,sem_df, on="Drug Regimen")

In [11]:
std_sem

,Tumor STD (mm3),Tumor SEM (mm3)
Drug Regimen,,
Capomulin,4.994774,0.329346
Ceftamin,6.268188,0.469821
Infubinol,6.567243,0.492236
Ketapril,8.279709,0.603860
Naftisol,8.134708,0.596466
Placebo,7.821003,0.581331
Propriva,6.622085,0.544332
Ramicane,4.846308,0.320955
Stelasyn,7.710419,0.573111


In [12]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen
summary=pd.DataFrame.merge(mean_median, std_sem, on="Drug Regimen")

# This method produces everything in a single groupby function
summary

,Tumor Mean (mm3),Tumor Median (mm3),Tumor STD (mm3),Tumor SEM (mm3)
Drug Regimen,,,,
Capomulin,40.675741,41.557809,4.994774,0.329346
Ceftamin,52.591172,51.776157,6.268188,0.469821
Infubinol,52.884795,51.820584,6.567243,0.492236
Ketapril,55.235638,53.698743,8.279709,0.603860
Naftisol,54.331565,52.509285,8.134708,0.596466
Placebo,54.033581,52.288934,7.821003,0.581331
Propriva,52.320930,50.446266,6.622085,0.544332
Ramicane,40.216745,40.673236,4.846308,0.320955
Stelasyn,54.233149,52.431737,7.710419,0.573111


## Bar and Pie Charts

In [20]:
# Generate a bar plot showing the total number of mice for each treatment throughout the course of the study using pandas. 


mice_count=df.groupby(["Drug Regimen"])['Mouse ID'].count()
mice_count

Drug Regimen
Capomulin    230
Ceftamin     178
Infubinol    178
Ketapril     188
Naftisol     186
Placebo      181
Propriva     148
Ramicane     228
Stelasyn     181
Zoniferol    182
Name: Mouse ID, dtype: int64

In [ ]:
# Generate a bar plot showing the total number of mice for each treatment throughout the course of the study using pyplot.



In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pandas



In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot



## Quartiles, Outliers and Boxplots

In [ ]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Start by getting the last (greatest) timepoint for each mouse


# Merge this group df with the original dataframe to get the tumor volume at the last timepoint


In [ ]:
# Put treatments into a list for for loop (and later for plot labels)


# Create empty list to fill with tumor vol data (for plotting)


# Calculate the IQR and quantitatively determine if there are any potential outliers. 

    
    # Locate the rows which contain mice on each drug and get the tumor volumes
    
    
    # add subset 
    
    
    # Determine outliers using upper and lower bounds
    

In [ ]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest


## Line and Scatter Plots

In [ ]:
# Generate a line plot of time point versus tumor volume for a mouse treated with Capomulin


In [ ]:
# Generate a scatter plot of mouse weight versus average tumor volume for the Capomulin regimen


## Correlation and Regression

In [ ]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen
